In [1]:
import os
import pandas as pd
import sqlite3

# Create the folders for your pipeline
os.makedirs('raw', exist_ok=True)
os.makedirs('processed', exist_ok=True)
os.makedirs('output', exist_ok=True)

In [2]:
import pandas as pd

raw_df = pd.read_csv('raw/ecommerce_dataset_updated.csv')

print(f"Total rows extracted: {len(raw_df)}")
raw_df.head()

Total rows extracted: 3660


,User_ID,Product_ID,Category,Price (Rs.),Discount (%),Final_Price(Rs.),Payment_Method,Purchase_Date
0,337c166f,f414122f-e,Sports,36.53,15,31.05,Net Banking,12-11-2024
1,d38a19bf,fde50f9c-5,Clothing,232.79,20,186.23,Net Banking,09-02-2024
2,d7f5f0b0,0d96fc90-3,Sports,317.02,25,237.76,Credit Card,01-09-2024
3,395d4994,964fc44b-d,Toys,173.19,25,129.89,UPI,01-04-2024
4,a83c145c,d70e2fc6-e,Beauty,244.80,20,195.84,Net Banking,27-09-2024


In [3]:
# Standardize column names (lowercase and underscores) [cite: 165]
raw_df.columns = [c.lower().replace(' ', '_').replace('.', '') for c in raw_df.columns]

# Handle missing values and duplicates [cite: 164]
clean_df = raw_df.drop_duplicates().fillna('Unknown')

In [5]:
# 1. Standardize names more aggressively to remove the ()
clean_df.columns = [c.lower().replace(' ', '_').replace('.', '').replace('(', '').replace(')', '') for c in clean_df.columns]

# 2. Check the names again to be 100% sure
print("Corrected Columns:", clean_df.columns)

# 3. Now the math will work perfectly
clean_df['discount_amount'] = clean_df['price_rs'] - clean_df['final_pricers']
clean_df['high_value_order'] = clean_df['final_pricers'] > 200

clean_df.head()

Corrected Columns: Index(['user_id', 'product_id', 'category', 'price_rs', 'discount_%',
       'final_pricers', 'payment_method', 'purchase_date'],
      dtype='object')


,user_id,product_id,category,price_rs,discount_%,final_pricers,payment_method,purchase_date,discount_amount,high_value_order
0,337c166f,f414122f-e,Sports,36.53,15,31.05,Net Banking,12-11-2024,5.48,False
1,d38a19bf,fde50f9c-5,Clothing,232.79,20,186.23,Net Banking,09-02-2024,46.56,False
2,d7f5f0b0,0d96fc90-3,Sports,317.02,25,237.76,Credit Card,01-09-2024,79.26,True
3,395d4994,964fc44b-d,Toys,173.19,25,129.89,UPI,01-04-2024,43.30,False
4,a83c145c,d70e2fc6-e,Beauty,244.80,20,195.84,Net Banking,27-09-2024,48.96,False


In [6]:
import sqlite3

# Connect to a database file (this creates it if it doesn't exist)
conn = sqlite3.connect('output/database.sqlite')

# Load the cleaned data into a table named 'ecommerce_sales'
clean_df.to_sql('ecommerce_sales', conn, if_exists='replace', index=False)

# Close the connection
conn.close()
print("ETL Loading Complete: database.sqlite has been created.")

ETL Loading Complete: database.sqlite has been created.


In [7]:
clean_df.to_csv('processed/processed_data.csv', index=False)